# Document tagging

*This notebook first appeared as a [blog post](//betatim.github.io/posts/bumping) on [Tim Head](//betatim.github.io)'s blog.*

*License: [MIT](http://opensource.org/licenses/MIT)*

*(C) 2016, Tim Head.*
*Feel free to use, distribute, and modify with the above attribution.*

In [2]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

In [3]:
from collections import Counter

import numpy as np
np.random.seed(3)
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

In [117]:
fname = '/Users/thead/Downloads/Train.csv'
df = pd.read_csv(fname, nrows=100000, index_col='Id', engine='c')

In [ ]:
df.head()

In [ ]:
df.Tags = df.Tags.map(lambda x: x.split())

In [ ]:
def encode_tags(tags, n_tags=40):
    tags_ = Counter()
    for v in tags:
        tags_.update(v)

    keys = list(sorted(v[0] for v in tags_.most_common(n_tags)))

    encoded = np.zeros((len(tags), n_tags))
    for i,row in enumerate(tags):
        for tag in row:
            if tag in keys:
                j = keys.index(tag)
                encoded[i, j] = 1
                
    return encoded
    
encode_tags(df.Tags[:10])

In [118]:
from html.parser import HTMLParser

class TextExtractor(HTMLParser):
    def __init__(self):
        super().__init__()
        self.in_code = False
        self.text = []
        
    def handle_starttag(self, tag, attrs):
        if tag == 'code':
            self.in_code = True

    def handle_endtag(self, tag):
        if tag == 'code':
            self.in_code = False

    def handle_data(self, data):
        if not self.in_code:
            self.text.append(data)


def clean_body(body):
    extractor = TextExtractor()
    extractor.feed(body)
    return ' '.join(extractor.text)


df['CleanBody'] = df.Body.map(clean_body)
        
tfidf = TfidfVectorizer(stop_words='english')

In [119]:
df['Text'] = df.apply(lambda x: x.Title +' '+x.CleanBody, axis=1)

In [120]:
df.head()

,Title,Body,Tags,CleanBody,Text
Id,,,,,
1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...,I'd like to check if an uploaded file is an im...,How to check if an uploaded file is an image w...
2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox,"In my favorite editor (vim), I regularly use c...",How can I prevent firefox from closing when I ...
3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning,I am import matlab file and construct a data f...,R Error Invalid type (list) for variable I am ...
4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding,"This is probably very simple, but I simply can...",How do I replace special characters in a URL? ...
5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents,"\n\n using this modify function, displays warn...",How to modify whois contact details? \n\n usin...


In [ ]:
tags = Counter()
for v in df.Tags:
    tags.update(v)
    
tags.most_common(40)

In [ ]:
len(tags.keys())

In [ ]:
plt.hist(df.Tags.apply(len), range=(0,5), bins=5)

In [ ]:
df.Tags.apply()

In [28]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

import pyLDAvis
pyLDAvis.enable_notebook()

ImportError: No module named 'pyLDAvis.sklearn'

In [106]:
#dataset = fetch_20newsgroups(shuffle=True, random_state=1,
#                             remove=('headers', 'footers', 'quotes'))


cats = ['talk.religion.misc', 'alt.atheism', 'comp.graphics', 'sci.med', 'sci.space']
dataset = fetch_20newsgroups(subset='train', categories=cats,
                             shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))

In [107]:
Counter(dataset.target)

Counter({0: 480, 1: 584, 2: 594, 3: 593, 4: 377})

In [108]:
dataset.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'sci.space', 'talk.religion.misc']

In [125]:
def norm(a):
    a = np.asarray(a)
    return a/(a.sum(axis=1)[:,np.newaxis])

def norm_(a):
    a = np.asarray(a)
    return a/a.sum()

vect = CountVectorizer(stop_words='english', max_df=0.95, min_df=2)
lda = LatentDirichletAllocation(n_topics=10)

docs = dataset['data']
docs = df.Text[:1000]

vectorised = vect.fit_transform(docs)
doc_topic_prob = lda.fit_transform(vectorised)

In [126]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    
print_top_words(lda, vect.get_feature_names(), 15)

Topic #0:
xml iphone api object want code application user development rows use exception app using entity
Topic #1:
javascript jquery use want using like code thread way html page create rails value thanks
Topic #2:
android page image problem code set file using method type make user facebook text css
Topic #3:
http php data com array 00 form using like 10 ve number code want string
Topic #4:
frac infty mathrm tests active session cdot integral int_ dx int_0 pdf question prove integration
Topic #5:
server database table sql windows using error access client connection id data install tables network
Topic #6:
java node function org nodes tree hibernate axis parent 1234 case graph attribute prefix length
Topic #7:
mode report random software crystal mac good restart emacs spanned numbers animation sleep nginx volume
Topic #8:
files width file swf utf detect scale height makefile segmentation windows repair rotation vista copied
Topic #9:
like using code use file way application want try

In [123]:
# topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency,
opts = dict(vocab=vect.get_feature_names(),
            doc_topic_dists=norm(doc_topic_prob),
            doc_lengths=np.array((vectorised != 0).sum(1)).squeeze(),
            topic_term_dists=norm(lda.components_),
            term_frequency=norm_(vectorised.sum(axis=0).tolist()[0]),)

In [124]:
import warnings
warnings.filterwarnings('ignore')
pyLDAvis.prepare(**opts, mds='tsne')

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
7      17.826084        1       1 -150.106447  -56.424528
9      15.141644        1       2   15.005294  -51.354965
2      13.769062        1       3  110.917525 -100.679145
1       9.886427        1       4   50.120055   33.324194
0       9.631546        1       5  -38.226027   61.058458
6       9.302145        1       6  -23.867423  167.020969
3       8.952511        1       7 -139.361305   66.965510
8       6.065348        1       8   87.107055  125.714708
5       5.205393        1       9  146.730341   11.797743
4       4.219841        1      10  -22.153704 -141.731195, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
229   Default  14.000000         http  14.000000  30.0000  30.0000
561   Default  12.000000          www  12.000000  29.0000  29.0000
84    Default  16.000000          com  16.000000  28.0000  28.0000
182   Default  24.000000         file  24.000000  27.0000  27.0000
386   Default   5.000000        query   5.000000  26.0000  26.0000
431   Default  15.000000       server  15.000000  25.0000  25.0000
526   Default  12.000000         user  12.000000  24.0000  24.0000
27    Default  14.000000  application  14.000000  23.0000  23.0000
258   Default   6.000000   javascript   6.000000  22.0000  22.0000
382   Default   4.000000     property   4.000000  21.0000  21.0000
263   Default   3.000000         json   3.000000  20.0000  20.0000
518   Default   3.000000         unit   3.000000  19.0000  19.0000
191   Default  12.000000    following  12.000000  18.0000  18.0000
418   Default   8.000000          run   8.000000  17.0000  17.0000
187   Default   5.000000      firefox   5.000000  16.0000  16.0000
234   Default   7.000000        image   7.000000  15.0000  15.0000
416   Default   5.000000         rows   5.000000  14.0000  14.0000
513   Default   8.000000         type   8.000000  13.0000  13.0000
113   Default  11.000000       create  11.000000  12.0000  12.0000
347   Default   6.000000         page   6.000000  11.0000  11.0000
80    Default   4.000000       client   4.000000  10.0000  10.0000
124   Default  14.000000         data  14.000000   9.0000   9.0000
532   Default   5.000000       values   5.000000   8.0000   8.0000
321   Default  13.000000         need  13.000000   7.0000   7.0000
455   Default   3.000000      sources   3.000000   6.0000   6.0000
81    Default  18.000000         code  18.000000   5.0000   5.0000
314   Default   3.000000       modify   3.000000   4.0000   4.0000
447   Default   3.000000         size   3.000000   3.0000   3.0000
117   Default   3.000000         ctrl   3.000000   2.0000   2.0000
104   Default   4.000000   controller   4.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
317   Topic10   0.401726     multiple   2.527860   1.3260  -5.4523
232   Topic10   0.421707        ideas   2.665322   1.3216  -5.4038
325   Topic10   0.972518          net   6.209762   1.3114  -4.5682
275   Topic10   0.398524    libraries   2.558643   1.3059  -5.4603
493   Topic10   0.399762        thank   2.579317   1.3010  -5.4572
241   Topic10   0.385824  information   2.498956   1.2971  -5.4927
13    Topic10   0.389374       adding   2.583778   1.2729  -5.4836
235   Topic10   0.396747       images   2.634603   1.2722  -5.4648
200   Topic10   0.389152    functions   2.672634   1.2385  -5.4841
161   Topic10   0.398190  environment   2.739888   1.2366  -5.4612
377   Topic10   0.387223     progress   2.668837   1.2350  -5.4891
215   Topic10   0.391133      greatly   2.727631   1.2232  -5.4790
130   Topic10   0.692197      default   5.148603   1.1588  -4.9082
5     Topic10   0.679650         able   5.535095   1.0681  -4.9265
199   Topic10   0.666929     function   6.188878   0.9375  -4.9454
103   Topic10   0.694210      control   6.677